#### FraudMachineLearning

In [ ]:
# importar os pacotes necessários
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
file_path = "https://media.githubusercontent.com/media/lmattos1603/TCC/main/dataset.csv"

# importar os dados para um dataframe

df = pd.read_csv(file_path)


In [ ]:
#observar as 5 primeiras entradas do nosso dataset
df.head()

In [ ]:
#observar o resumo estatístico do dataset
df.describe()

In [ ]:
# ordenar em ordem decrescente as variáveis por seus valores ausentes
(df.isnull().sum() / df.shape[0]).sort_values(ascending=False)

In [ ]:
# plotar o histograma das variáveis numéricas
df.hist(bins = 5, figsize=(15,10));

In [ ]:
# balanceamento das classes
print(df.Class.value_counts())
print("\nFraudes representam {:.2f}% do dataset.\n".format((df[df.Class == 1].shape[0] / df.shape[0]) * 100))

# plotar gráfico de barras para as Classes
fig, ax = plt.subplots()
sns.countplot('Class', data=df, ax=ax)
ax.set_title('Distribuição das Classes\n (0: Normal || 1: Fraude)', fontsize=14)
plt.plot()

#### Distribuição no tempo

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12,6))

num_bins = 40

ax[0].hist(df.Time[df.Class == 0], bins=num_bins)
ax[0].set_title('Normal')

ax[1].hist(df.Time[df.Class == 1], bins=num_bins)
ax[1].set_title('Fraude')

plt.xlabel('Tempo (segundos)')
plt.ylabel('Transações')
plt.tight_layout()

In [ ]:
#instanciando o objeto
fig, ax = plt.subplots(figsize = (6,12))
#gerando dados para o gráfico
sns.boxplot(df.Class, df.Amount, showmeans=True, ax=ax)
plt.ylim((-20,400))
plt.xticks([0,1], ['Normal', 'Fraude'])
ax.set_title('Distribuição das Classes\n (Normal || Fraude)')
#plotando o gráfico
plt.tight_layout()

#### Densidade de distribuição

In [ ]:
column_names = df.drop(['Class', 'Amount', 'Time'], axis=1).columns
num_plots = len(column_names)
df_class_0 = df[df.Class == 0]
df_class_1 = df[df.Class == 1]

fig, ax = plt.subplots(nrows=7, ncols=4, figsize=(18,18))
fig.subplots_adjust(hspace=1, wspace=1)

idx = 0
for col in column_names:
    idx += 1
    plt.subplot(7, 4, idx)
    sns.kdeplot(df_class_0[col], label="Class 0", shade=True)
    sns.kdeplot(df_class_1[col], label="Class 1", shade=True)
    plt.title(col, fontsize=10)
plt.tight_layout()

## Preparação dos Dados

In [ ]:
#criar um novo dataset por precaução
df_clean = df.copy()

Importar as bibliotecas necessárias

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score

Padrozinar as variáveis `Time` e `Amount`

In [ ]:
std_scaler = StandardScaler()
df_clean['std_time'] = std_scaler.fit_transform(df_clean['Time'].values.reshape(-1,1))
df_clean['std_amount'] = std_scaler.fit_transform(df_clean['Amount'].values.reshape(-1, 1))

df_clean.drop(['Amount', 'Time'], axis=1, inplace=True) 

In [ ]:
df_clean.head()

### Dividindo o dataset (treino e teste)

In [ ]:
#separando as variáveis
X = df_clean.drop('Class', axis=1)
y = df_clean['Class']

#dividindo o dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

### Balanceando o conjunto de dados

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rand = RandomUnderSampler()
X_rand, y_rand = rand.fit_sample(X_train, y_train)

sns.countplot(y_rand);


In [ ]:
corr = X_train.corr()
corr_rand = pd.DataFrame(X_rand).corr()

fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (12,6))
fig.suptitle ('Matriz de Correlação para o Dataset')

sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns,
            linewidths=.3, cmap="RdBu", ax=ax[0])
ax[0].set_title('Desbalanceado')

sns.heatmap(corr_rand, xticklabels=corr_rand.columns, yticklabels=corr_rand.columns,
            linewidths=.3, cmap="RdBu", ax=ax[1])
ax[1].set_title('Balanceado')
#plt.savefig('CC-Fraud-correlaction.jpg')
plt.show;

## Modelo de Machine Learning

### Sklearn:

In [ ]:
#1. Escolher o modelo
from sklearn.linear_model import LogisticRegression

#2. Instanciar o modelo
np.random.seed(2)
model_lr = LogisticRegression()

#3. Fit do modelo
model_lr.fit(X_rand, y_rand)

#5. Fazer previsões em cima dos novos dados
y_pred_lr = model_lr.predict(X_test)
y_prob_lr = model_lr.predict_proba(X_test)

### Árvore de Decisão:

In [ ]:
#1. Escolha do modelo
from sklearn.tree import DecisionTreeClassifier

#2. Instanciar o modelo e escolher os hyperparametros
model_tree = DecisionTreeClassifier(max_depth=4, criterion="entropy")

#3. Fit do modelo
model_tree.fit(X_rand, y_rand)

#Fazer previsões em cima dos novos dados
y_pred_tree = model_tree.predict(X_test)

In [ ]:
from sklearn import tree
plt.figure(figsize=(18,8))#ajustar o tamanho da figura
tree.plot_tree(model_tree); #utilizando a própria biblioteca sklearn podemos plotar a árvore.
#plt.savefig('tree.jpg')


### Avaliar o desempenho utilizando Sklearn

#### Matriz de Confusão Analítica

In [ ]:
print("Utilizando Regressão Logística: \n" )
print(classification_report(y_test, y_pred_lr))

print("Utilizando Árvore de Decisão: \n")
print(classification_report(y_test, y_pred_tree))

#### Matriz de Confusão Gráfica

In [ ]:
confusion_matrix_lr = confusion_matrix(y_test, y_pred_lr) #gerar matriz de confusão para regressão logística
confusion_matrix_tree = confusion_matrix(y_test, y_pred_tree)  #gerar matriz de confusão para árovre de decisão

fig, (ax,ax2) = plt.subplots(ncols=2, figsize=(12,6))
fig.suptitle('Matriz de Confusão')

sns.heatmap(confusion_matrix_lr, ax=ax, annot=True, cmap=plt.cm.copper)
ax.set_title('Regressão Logísitca',fontsize='medium')
ax.set_ylabel = ('Verdadeiros')
ax.set_xlabel = ('Positivos')

sns.heatmap(confusion_matrix_tree, ax=ax2, annot=True, cmap=plt.cm.copper)
ax2.set_title('Árvore de Decisão',fontsize='medium')
ax2.set_ylabel = ('Verdadeiros')
ax2.set_xlabel = ('Positivos')
#plt.savefig('CC-Fraud-Confusion.jpg')
plt.show(fig)

#### AUC ROC

In [ ]:
# imprimir a acurácia do modelo
print("Acurácia utilizando regressão logística: {:.4f}".format(accuracy_score(y_test, y_pred_lr)))
print("Acurácia utilizando árvore de decisão: {:.4f}\n".format(accuracy_score(y_test, y_pred_tree)))

# imprimir a área sob da curva
print("\nAUC ROC utilizando regressão logística: {:.4f}".format(roc_auc_score(y_test, y_pred_lr)))
print("AUC ROC utilizando árvore de decisão: {:.4f}\n".format(roc_auc_score(y_test, y_pred_tree)))

#### Construindo o Protótipo

In [ ]:
import pickle

In [ ]:
!pip install pyngrok
!pip install streamlit

In [ ]:
arquivo = open("dump.pkl", "wb")
pickle.dump(model_lr, arquivo)
arquivo.close()

In [ ]:
!streamlit run App.py &>dev>null&

In [ ]:
from pyngrok import ngrok
url = ngrok.connect(8501)
url

In [ ]:
ngrok.kill()